In [1]:
import findspark
findspark.init('/home/danielf/spark-3.3.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName('lrproject').getOrCreate()

23/02/27 11:30:16 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/27 11:30:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 11:30:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/27 11:30:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/27 11:30:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/27 11:30:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
path = '/home/danielf/Desktop/pyspark-course-udemy/Spark_for_Machine_Learning/Logistic_Regression/customer_churn.csv'
df = spark.read.csv(path, header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.select(['Names', 'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Onboard_date', 'Location', 'Company', 'Churn']).show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [10]:
df.corr('Account_Manager', 'Churn')

0.07061077173214911

In [22]:
df.corr('Total_Purchase', 'Churn')

0.024030894518220495

In [95]:
#Age, #Account_Manager
final_df = df.select(['Age', 'Account_Manager', 'Total_Purchase', 'Years', 'Num_Sites', 'Churn'])

In [96]:
#df.select(['Company']).distinct().count()
#df.select(['Company']).count()

In [97]:
#company_indexer = StringIndexer(inputCol='Company', outputCol='CompanyIndex')
#company_encoder = OneHotEncoder(inputCol='CompanyIndex', outputCol='CompanyVec')

In [98]:
assembler = VectorAssembler(inputCols=['Age', 'Account_Manager', 'Total_Purchase', 'Years', 'Num_Sites'], outputCol='features')

In [99]:
lr = LogisticRegression(featuresCol='features', labelCol='Churn')

In [100]:
pipeline = Pipeline(stages=[assembler, lr])

In [101]:
train, test = final_df.randomSplit([0.7, 0.3])
#test.show()

In [102]:
fit_lr = pipeline.fit(train)

In [103]:
results = fit_lr.transform(test)
#results.head()

In [104]:
myeval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [105]:
auc = myeval.evaluate(results)

In [106]:
auc

0.7128851540616247

In [108]:
results.select('Churn', 'prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
+-----+----------+
only showing top 20 rows

